Return the top 200 most recently posted jobs on the portal, and each job posting should
at least contain the Job Title, Job ID, URL, Company, Salary Range (if applicable),
and any other information that you deem appropriate. The output should be an
.xlsx file 

# Beautiful Soup

Import libraries

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# to Excel file

In [44]:
job_200_list = []

Function to scrape jobs from one page

In [45]:
def get_jobs_from_page(page_num):
    URL = "https://sg.jobstreet.com/jobs?page={page_num}&sortmode=ListedDate"
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")
    job_list = []
    job_cards = soup.find_all(attrs={"data-testid": "job-card"})
    
    for job in job_cards:
        
        # Extract job title 
        job_title_text = job.get('aria-label')
        
        #Extract job id
        job_id = job.get('data-job-id')

        #Extract company
        company_tag = job.find(attrs={'data-automation': 'jobCompany'})
        company_name_text = company_tag.text.strip() if company_tag else 'N/A'

        #Extract salary range
        salary_tag = job.find(attrs={'data-automation': 'jobSalary'})
        salary_range = salary_tag.text.strip() if salary_tag else 'N/A'
        
        # Extract URL
        #url_tag = job.find(attrs={'data-automation': 'job-list-view-job-link'})
        #job_url = url_tag['href'] if url_tag else 'N/A'

        # Extract Job Type or brief description
        job_type_tag = job.find('p')
        job_type = job_type_tag.text.strip() if job_type_tag else 'N/A'

        # Extract Job Locations
        location_tags = job.find_all(attrs={'data-automation': 'jobCardLocation'})
        job_locations = [loc.get_text(strip=True) for loc in location_tags]

        # Extract Listing Date
        listing_date_tag = job.find(attrs={'data-automation': 'jobListingDate'})
        listing_date = listing_date_tag.text.strip() if listing_date_tag else 'N/A'

        # Append the job data to the list
        job_list.append({
            'Job Title': job_title_text,
            'Job ID': job_id,
            #'URL': job_url,
            'Company': company_name_text,
            'Salary Range': salary_range,
            'Job Type': job_type,
            'Job Locations': job_locations,
            'Listing Date': listing_date
        })
    return job_list

Function to get jobs from beyond page 1

In [46]:
page_num = 1
total_jobs = 0
while total_jobs <200:
    jobs = get_jobs_from_page(page_num)
    if jobs == False:
        print("No jobs found on this page or unable to retrieve jobs.")
        break
    
    # Add the new jobs to the main job_data list
    job_200_list.extend(jobs)
    
    total_jobs = len(job_200_list)
    if total_jobs >200: break
    
    page_num += 1

In [47]:
job_200_list
len(job_200_list)

224

Convert to df, cut off 200 jobs and save to .xlsx file

In [48]:
df = pd.DataFrame(job_200_list)
df_first_200 = df.head(200)
len(df_first_200)
df.head(5)

,Job Title,Job ID,Company,Salary Range,Job Type,Job Locations,Listing Date
0,Senior Planning Engineer ( Waterfront Project ),81997354,China Jingye Engineering Corporation Limited (...,N/A,This is a Full time job,"[Sentosa, ,Central Region]",29m ago
1,Security Systems Project Director,81997351,CERTIS GROUP - GHR,N/A,This is a Full time job,"[Commonwealth, ,Central Region]",29m ago
2,Store Assistant,81997313,SIMCC PTE LTD,"$2,000 – $2,500 per month",This is a Full time job,[Central Region],34m ago
3,PUBLIC RELATIONS OFFICER – CONSTRUCTION,81997306,People Profilers Pte Ltd,"$2,500 – $3,100 per month",This is a Full time job,[Central Region],37m ago
4,Executive / Senior Executive - Retail Campaign...,81997291,Lagardere Travel Retail Singapore Pte Ltd,N/A,This is a Full time job,"[Paya Lebar, ,East Region]",41m ago


In [49]:
df_first_200.to_excel('cyx_jobstreet_jobs.xlsx', index=False)

# to Google Sheets

In [50]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [51]:
# Authenticate and authorize with Google Sheets API using OAuth2 credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/Users/Admin/Downloads/Webscraping/solid-ruler-450505-i5-5edbb392240a.json", scope)
client = gspread.authorize(creds)

# Open the Google Sheet by url
worksheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1iQaMeo8uVr-wsolRha1iiSq4cZQsjwlwAYFQXSdYnCE/edit?gid=0#gid=0').get_worksheet(0)

# Modify the location column to combine its values into a single string
df_first_200['Job Locations'] = df_first_200['Job Locations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

# Write the DataFrame to the Google Sheets (starting at row 1, column 1)
worksheet.update([df_first_200.columns.values.tolist()] + df_first_200.values.tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_21584\4021521300.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_200['Job Locations'] = df_first_200['Job Locations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


{'spreadsheetId': '1iQaMeo8uVr-wsolRha1iiSq4cZQsjwlwAYFQXSdYnCE',
 'updatedRange': 'Sheet1!A1:G201',
 'updatedRows': 201,
 'updatedColumns': 7,
 'updatedCells': 1407}

# Selenium

import libraries

In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd


Initialise Webdriver

In [53]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [54]:
def get_jobs_from_page(page_num):
    URL = f"https://sg.jobstreet.com/jobs?page={page_num}&sortmode=ListedDate"
    
    # Open the webpage with Selenium
    driver.get(URL)
    
    # Give the page some time to load (you can use WebDriverWait for a more efficient wait)
    time.sleep(3)
    
    # Find all the job cards on the page
    job_list = []
    job_cards = driver.find_elements(By.CSS_SELECTOR, '[data-testid="job-card"]')
    
    # Extract information for each job card
    for job in job_cards:
        # Extract job title
        job_title_text = job.get_attribute('aria-label')
        
        # Extract job ID
        job_id = job.get_attribute('data-job-id')
        
        # Extract company name
        company_tag = job.find_element(By.CSS_SELECTOR, '[data-automation="jobCompany"]')
        company_name_text = company_tag.text.strip() if company_tag else 'N/A'
        
        # Extract salary range
        salary_tag = WebDriverWait(driver, 10).until(
             EC.presence_of_element_located((By.CSS_SELECTOR, '[data-automation="jobSalary"]'))
             )
        salary_range = salary_tag.text.strip() if salary_tag else 'N/A'
        
        # Extract the job URL
        url_tag = job.find_element(By.CSS_SELECTOR, '[data-automation="job-list-view-job-link"]')
        full_job_url = url_tag.get_attribute('href') if url_tag else 'N/A'
        
        # Extract Job Type or brief description
        job_type_tag = job.find_element(By.TAG_NAME, 'p')
        job_type = job_type_tag.text.strip() if job_type_tag else 'N/A'
        
        # Extract Job Locations
        location_tags = job.find_elements(By.CSS_SELECTOR, '[data-automation="jobCardLocation"]')
        job_locations = [loc.text.strip() for loc in location_tags]
        
        # Extract Listing Date
        listing_date_tag = job.find_element(By.CSS_SELECTOR, '[data-automation="jobListingDate"]')
        listing_date = listing_date_tag.text.strip() if listing_date_tag else 'N/A'

        # Append the job data to the list
        job_list.append({
            'Job Title': job_title_text,
            'Job ID': job_id,
            'URL': full_job_url,
            'Company': company_name_text,
            'Salary Range': salary_range,
            'Job Type': job_type,
            'Job Locations': job_locations,
            'Listing Date': listing_date
        })
    
    return job_list


In [55]:
# Initialize the list to hold all job data
job_200_list = []
total_jobs = 0
page_num = 1

# Loop through pages and scrape jobs
while total_jobs < 200:
    jobs = get_jobs_from_page(page_num)
    
    if not jobs:
        print("No jobs found on this page or unable to retrieve jobs.")
        break
    
    # Add the new jobs to the main job_data list
    job_200_list.extend(jobs)
    
    total_jobs = len(job_200_list)
    
    if total_jobs >= 200:
        break
    
    # Go to the next page
    page_num += 1

# Create a DataFrame from the collected job data
df = pd.DataFrame(job_200_list)

# Get the first 200 rows (in case we scraped more than 200)
df_first_200 = df.head(200)

In [56]:
df_first_200.head(5)

,Job Title,Job ID,URL,Company,Salary Range,Job Type,Job Locations,Listing Date
0,Senior Planning Engineer ( Waterfront Project ),81997354,https://sg.jobstreet.com/job/81997354?type=sta...,China Jingye Engineering Corporation Limited (...,"$2,000 – $2,500 per month",This is a Full time job,"[Sentosa, , Central Region]",30m ago
1,Security Systems Project Director,81997351,https://sg.jobstreet.com/job/81997351?type=sta...,CERTIS GROUP - GHR,"$2,000 – $2,500 per month",This is a Full time job,"[Commonwealth, , Central Region]",30m ago
2,Store Assistant,81997313,https://sg.jobstreet.com/job/81997313?type=sta...,SIMCC PTE LTD,"$2,000 – $2,500 per month",This is a Full time job,[Central Region],34m ago
3,PUBLIC RELATIONS OFFICER – CONSTRUCTION,81997306,https://sg.jobstreet.com/job/81997306?type=sta...,People Profilers Pte Ltd,"$2,000 – $2,500 per month",This is a Full time job,[Central Region],37m ago
4,Executive / Senior Executive - Retail Campaign...,81997291,https://sg.jobstreet.com/job/81997291?type=sta...,Lagardere Travel Retail Singapore Pte Ltd,"$2,000 – $2,500 per month",This is a Full time job,"[Paya Lebar, , East Region]",41m ago


Write the DataFrame to an Excel file

In [57]:
df_first_200.to_excel('cyx_jobstreet_jobs_selenium.xlsx', index=False)

Write to Google Sheets

In [58]:
# Authenticate and authorize with Google Sheets API using OAuth2 credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/Users/Admin/Downloads/Webscraping/solid-ruler-450505-i5-5edbb392240a.json", scope)
client = gspread.authorize(creds)

# Open the Google Sheet by url
worksheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1iQaMeo8uVr-wsolRha1iiSq4cZQsjwlwAYFQXSdYnCE/edit?gid=975270541#gid=975270541').get_worksheet(1)

# Modify the location column to combine its values into a single string
df_first_200['Job Locations'] = df_first_200['Job Locations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

# Write the DataFrame to the Google Sheets (starting at row 1, column 1)
worksheet.update([df_first_200.columns.values.tolist()] + df_first_200.values.tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_21584\3751486130.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_200['Job Locations'] = df_first_200['Job Locations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)


{'spreadsheetId': '1iQaMeo8uVr-wsolRha1iiSq4cZQsjwlwAYFQXSdYnCE',
 'updatedRange': 'Sheet2!A1:H201',
 'updatedRows': 201,
 'updatedColumns': 8,
 'updatedCells': 1608}

In [59]:
driver.quit()